In [ ]:
# 1. 数据的收集与预处理

## 1.1.  数据收集与可视化分析

In [ ]:
import numpy as np 
import pandas as pd 

df_bank = pd.read_csv('/kaggle/input/bank-customer/BankCustomer.csv')
df_bank.head()
# Tenure：已经成为客户的年头。  ProductsNo：拥有的产品数量。
# HasCard：是否有信用卡。      ActiveMember：是否为活跃用户。
# Credit：信用评级。          AccountBal：银行存款余额。        Salary：薪水。

In [ ]:
# 显示数据的分布情况：
import matplotlib.pyplot as plt
import seaborn as sns
features = ['City', 'Gender', 'Age', 'Tenure',
                    'ProductsNo', 'HasCard', 'ActiveMember', 'Exited']
fig = plt.subplots(figsize=(15, 15))
for i, j in enumerate(features):
    plt.subplot(4, 2, i + 1)
    plt.subplots_adjust(hspace=1.0)
    sns.countplot(x=j, data=df_bank)
    plt.title("No. of costumers")

从图中大概看得出，北京的客户最多，男女客户比例大概一致，年龄和客户数量呈现正态分布（钟形曲线，中间高两边低）。

对这个数据集，我们主要做以下3方面的清理工作。
1. **性别**。这是一个二元类别特征，需要转换为0/1代码格式进行读取处理（机器学习中的文本格式数据都要转换为数字代码）。
2. **城市**。这是一个多元类别特征，应把它转换为多个二元类别哑变量。
3. **姓名**这个字段对于客户流失与否的预测应该是完全不相关的，可以在进一步处理之前忽略。

## 1.2. 数据集清洗与规范化

In [ ]:
# 数据清理：
# 把二元类别文本数字化
df_bank['Gender'].replace("Female", 0, inplace = True)
df_bank['Gender'].replace("Male", 1, inplace = True)
# 显示数字类别
print("Gender unique values: ", df_bank['Gender'].unique())

# 把多元类别转换成多个二元哑变量，然后贴回原始数据集
d_city = pd.get_dummies(df_bank['City'], prefix="City")
df_bank = [df_bank, d_city]
df_bank = pd.concat(df_bank, axis=1)
df_bank.head()

## 1.3. 拆分数据集为训练集、测试集、验证集

In [ ]:
# 构建特征和标签集合
X = df_bank.drop(['Name', 'Exited', 'City'], axis=1)
y = df_bank['Exited']

# 用标准方法拆分数据集为测试集和训练集
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train.head()

# 2. 构建神经网络(ANN)模型
## 2.1 构建 Sequential ANN 模型

In [ ]:
# 1. 用序贯模型构建网络
import keras
from keras.models import Sequential
from keras.layers import Dense

ann = Sequential()
ann.add(Dense(units=12, input_dim=12, activation='relu')) # 添加输入层
ann.add(Dense(units=24,  activation='relu'))    # 添加隐层
ann.add(Dense(units=1, activation='sigmoid'))   # 添加输出层
ann.summary()

## 2.2. 编译网络

In [ ]:
ann.compile(optimizer='adam',             # 优化器
            loss = 'binary_crossentropy', # 损失函数
            metrics = ['acc'])            # 评估指标

下面开始训练刚才编译好的神经网络

In [ ]:
# 编译神经网络，指定优化器，损失函数，以及评估标准
# 通过history变量把训练过程中的信息保存下来，留待以后分析
history = ann.fit(X_train, y_train,# 指定训练集
                  epochs=30,       # 指定训练轮次
                  batch_size=64,   # 指定数据批量
                  validation_data=(X_test, y_test))   # 指定验证集,这里为了简化模型，直接用测试集数据进行验证

由上面结果可见，使用顺序ANN网络模型的识别准确率为 71.25%